In [76]:
import pandas as pd
import numpy as np
import hashlib
from itertools import combinations
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from itertools import product

In [10]:
class SimpleCluster:
    def __init__(self,x):
        self.x = x
        
    def __hash__(self):
        return int(hashlib.md5(''.join(self.x).encode()).hexdigest(), 16)

def get_hash(plist):
    c = SimpleCluster(plist)
    return hash(c)

In [102]:
clusters = []
cluster_dict = {}

with open("/afs/csail/u/s/samsl/Work/Software/philharmonic/results/Pdam.clusters.csv","r") as cluster_fi:
# with open("Pdam.recipe_clusters.csv","r") as cluster_fi:
    for line in cluster_fi:
        c = line.strip().split(",")
        clusters.append(c)
        cluster_dict[get_hash(c)] = c

In [103]:
df = pd.read_excel("Connelly2022_SupplementaryData.xlsx", sheet_name="Table_S4A_DEGs_coral", header=1, index_col=0)

In [104]:
df = df.rename({
    "DE_Heat": "DE_Heat_control",
    "DE_Antibiotics": "DE_Antibiotics_control",
    "DE_Antibiotics.Heat": "DE_Antibiotics.Heat_control",
},axis=1)

In [105]:
cluster_diffexp = {}

for clk, clust in cluster_dict.items():
    cluster_prots = set([i.rstrip("-RA") for i in clust])
    cluster_prots = set.intersection(cluster_prots, df.index)
    cluster_diffexp[clk] = df.loc[list(cluster_prots)]

In [106]:
diffexp_df.head()

,log2FoldChange_Heat_control,log2FoldChange_Antibiotics_control,log2FoldChange_Antibiotics.Heat_control,padj_Heat_control,padj_Antibiotics_control,padj_Antibiotics.Heat_control,DE_Heat_control,DE_Antibiotics_control,DE_Antibiotics.Heat_control,Gene_Name,Gene_Info,GO_Terms_IPS,GO_Terms_EM,KO_Terms_KAAS,KO_Terms_EM,KOG_Class_A,KOG_Class,Pfam_Domains
ID,,,,,,,,,,,,,,,,,,
pdam_00023241,0.084789,0.305267,0.441482,0.776924,0.158868,0.008966,NaN,NaN,Upregulated,TSPAN5,Tspan33: Tetraspanin-33 (Mus musculus),GO:0016021,"GO:0000003,GO:0000151,GO:0001667,GO:0003006,GO...",NaN,"K10303,K17345",I,Lipid transport and metabolism,PF00335
pdam_00005331,-0.239801,-0.218533,-0.462572,0.069150,0.083830,0.000003,NaN,NaN,Downregulated,PLD3,pld3: Phospholipase D3 (Xenopus tropicalis),GO:0003824,"GO:0000003,GO:0002119,GO:0002164,GO:0003006,GO...",K16860,"K16860,K16861",I,Lipid transport and metabolism,"PF00614, PF13918"


In [107]:
conditions = ["Heat_control", "Antibiotics_control", "Antibiotics.Heat_control"]
p_thresh = 0.05

cluster_de_df = []

for clk, diffexp_df in tqdm(cluster_diffexp.items(),total=len(cluster_diffexp)):
    cluster_row = [clk]
    for cond in conditions:
        # log2FoldChange
        cluster_row.append(np.median(diffexp_df[f"log2FoldChange_{cond}"]))
        # padj
        cluster_row.append(np.sum(diffexp_df[f"padj_{cond}"] < p_thresh))
        # DE_Up
        cluster_row.append(np.sum(diffexp_df[f"DE_{cond}"] == "Upregulated"))
        # DE_Down
        cluster_row.append(np.sum(diffexp_df[f"DE_{cond}"] == "Downregulated"))
    cluster_de_df.append(cluster_row)
        
cluster_de_df = pd.DataFrame(cluster_de_df)
cluster_de_df.columns = ["key"] + [f"{i}_{j}" for i,j in product(conditions, ["log2FoldChange","padj","DE_Up","DE_Down"])]
cluster_de_df = cluster_de_df.set_index("key")

 81%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████                               | 472/582 [00:01<00:00, 394.80it/s]/data/cb/samsl/mambaforge/envs/philharmonic/lib/python3.12/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/data/cb/samsl/mambaforge/envs/philharmonic/lib/python3.12/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 582/582 [00:01<00:00, 389.01it/s]


In [108]:
cluster_de_df.to_csv("connelly2022_DE_perCluster_preRecipe.csv")

---

In [92]:
x = []
with open("/afs/csail/u/s/samsl/Work/Software/philharmonic/results/Pdam.clusters.csv","r") as f:
    for line in f:
        x.append(line.strip().split(","))

In [101]:
for xi, ci in zip(x, clusters):
    if xi != ci:
        print('mismatch', len(xi), len(ci))
        break

mismatch 29 49


In [97]:
x[0] == clusters[0]

True

In [94]:
len(x)

582

In [90]:
test_df.head()

,pdam_00001358-RA,pdam_00011541-RA,pdam_00016978-RA,pdam_00010849-RA,pdam_00016673-RA,pdam_00012878-RA,pdam_00008920-RA,pdam_00005572-RA,pdam_00015036-RA,pdam_00019926-RA,...,pdam_00004892-RA,pdam_00006246-RA,pdam_00003083-RA,pdam_00000191-RA,pdam_00001696-RA,pdam_00012489-RA,pdam_00008569-RA,pdam_00008873-RA,pdam_00019316-RA,pdam_00004893-RA
0,pdam_00006668-RA,pdam_00000181-RA,pdam_00002493-RA,pdam_00001975-RA,pdam_00021628-RA,pdam_00011827-RA,pdam_00009749-RA,pdam_00019784-RA,pdam_00000438-RA,pdam_00017417-RA,...,pdam_00017319-RA,pdam_00009104-RA,pdam_00011586-RA,pdam_00008481-RA,pdam_00013401-RA,pdam_00016274-RA,pdam_00013036-RA,pdam_00003204-RA,pdam_00011288-RA,pdam_00009471-RA
1,pdam_00008048-RA,pdam_00003654-RA,pdam_00019700-RA,pdam_00015353-RA,pdam_00013724-RA,pdam_00015010-RA,pdam_00004852-RA,pdam_00010718-RA,pdam_00015268-RA,pdam_00022708-RA,...,pdam_00005891-RA,pdam_00012819-RA,pdam_00002739-RA,pdam_00016471-RA,pdam_00021429-RA,pdam_00014952-RA,pdam_00001129-RA,pdam_00021997-RA,pdam_00020869-RA,pdam_00006972-RA
2,pdam_00012920-RA,pdam_00017590-RA,pdam_00021528-RA,pdam_00003093-RA,pdam_00006493-RA,pdam_00020754-RA,pdam_00011156-RA,pdam_00017660-RA,pdam_00019359-RA,pdam_00004601-RA,...,pdam_00025086-RA,pdam_00010284-RA,pdam_00015491-RA,pdam_00007993-RA,pdam_00019824-RA,pdam_00010969-RA,pdam_00001729-RA,pdam_00001586-RA,pdam_00001133-RA,pdam_00002351-RA
3,pdam_00023352-RA,pdam_00014065-RA,pdam_00025722-RA,pdam_00001204-RA,pdam_00004533-RA,pdam_00016700-RA,pdam_00014668-RA,pdam_00011177-RA,pdam_00012833-RA,pdam_00020760-RA,...,pdam_00018580-RA,pdam_00015183-RA,pdam_00019181-RA,pdam_00010554-RA,pdam_00013785-RA,pdam_00018071-RA,pdam_00015184-RA,pdam_00010757-RA,pdam_00009413-RA,pdam_00004375-RA
4,pdam_00023701-RA,pdam_00012332-RA,pdam_00000845-RA,pdam_00020355-RA,pdam_00018181-RA,pdam_00017707-RA,pdam_00011742-RA,pdam_00017004-RA,pdam_00010291-RA,pdam_00012180-RA,...,pdam_00023355-RA,pdam_00000555-RA,pdam_00000070-RA,pdam_00013527-RA,pdam_00008977-RA,pdam_00010292-RA,pdam_00024567-RA,pdam_00014111-RA,pdam_00007011-RA,pdam_00007512-RA
